# Технології ETL з бібліотекою `scikit-learn`

## УВЕДЕННЯ В SCIKIT-LEARN

### Загальна інформація

Розроблено для побудови та аналізу класичних моделей машинного навчання [(Докладніше)](https://en.wikipedia.org/wiki/Scikit-learn)
- [Офіційний сайт](https://scikit-learn.org/stable/)
- [Сайт на GitHub](https://github.com/scikit-learn/scikit-learn)

Альтернативи:
- [Tensorflow](https://www.tensorflow.org)
- [Pytorch](https://pytorch.org)
- [Keras](https://keras.io) ...

Доступна через [API](https://scikit-learn.org/stable/modules/classes.html#) з мови Python (або С/С++)

### Інструментарій для роботи з данними

Має потужні інструменти для [Data Transformation](https://scikit-learn.org/stable/data_transforms.html#data-transforms)

Автоматизують наступні функції
- [очищення](https://scikit-learn.org/stable/modules/unsupervised_reduction.html)
- [вилучення](https://scikit-learn.org/stable/modules/feature_extraction.html)
- [збагачення](https://scikit-learn.org/stable/modules/impute.html)
- [предобробка](https://scikit-learn.org/stable/modules/preprocessing.html)

Для інтеграції цих (та інших) процесів передбачена [`конвереїзація`](https://scikit-learn.org/stable/modules/compose.html)


## ТЕОРЕТИЧНА ЧАСТИНА ТА ПРИКЛАДИ

### для проведення експеріментів та перевірки гіпотез в `scilearn` є модуль моделювання датасетів [sklearn.datasets](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets)

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [ ]:
wine_bunch = datasets.load_wine(as_frame=True)

In [ ]:
df = wine_bunch['frame']

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.loc[df['target'] == 0,'target'] = 'west'
df.loc[df['target'] == 1,'target'] = 'east'
df.loc[df['target'] == 2,'target'] = 'nord'

In [ ]:
df['target'].unique()

In [ ]:
df.rename(columns={'target': 'region'}, inplace=True)
df.columns

In [ ]:
X, y = df.drop(columns=['alcohol']), df['alcohol']

In [ ]:
X.head(3)

### додамо синтетичний `шум' 

In [ ]:
from add_noise import add_feauture_err

In [ ]:
# деякі значення можуть бути пропущені
X = add_feauture_err(df=X, column_name='region', err_level=.05, err_filler=np.nan)
X = add_feauture_err(df=X, column_name='color_intensity', err_level=.10, err_filler=np.nan)

# або вочевидь невірні
X = add_feauture_err(df=X, column_name='magnesium', err_level=.05, err_filler=100000.0)

In [ ]:
X.describe()

In [ ]:
X.info()

### ПОШУК ТА ОБРОБКА ВИКИДІВ (Outliers/Anomaly Detection)

В продашені та на великих об'ємах використовується алгоритм __LOF__ ([_Local Outlier Factor_](https://scikit-learn.org/stable/auto_examples/neighbors/plot_lof_outlier_detection.html))

In [ ]:
mean_std = X.describe().loc[['mean', 'std']].T

In [ ]:
mean_std

In [ ]:
mean_std['std_minus_mean'] = mean_std['std'] - mean_std['mean']
mean_std

In [ ]:
# використаємо правило 2-х сігм для показчика `magnesium`
two_sigma = 2 * mean_std.loc['magnesium']['std']
two_sigma

In [ ]:
# замінити всі значення `magnesium` на медіану 
X.loc[df['magnesium'] > two_sigma] = np.median(X['magnesium'])

### заміщення пропусків

[Детальніше](https://scikit-learn.org/stable/modules/impute.html)

In [ ]:
X.describe().loc['count']

In [ ]:
X['color_intensity'].isna()

In [ ]:
# замінимо NaN на медіану
X['color_intensity'].fillna(value=X['color_intensity'].median(), inplace=True)

In [ ]:
# для строкового показчика - замінимо найближчим
X['region'].fillna(method='bfill', inplace=True)

In [ ]:
X.info()

### Стандартизація та масштабування числових показчиків

[Докладніше](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling)

In [ ]:
# відбір показчиків за їх типом
from sklearn.compose import make_column_selector
numeric_columns_selector = make_column_selector(dtype_include='float64')
X_numeric_columns = numeric_columns_selector(X)
X_numeric_columns

In [ ]:
X_numeric = X[X_numeric_columns]
X_numeric.info()

In [ ]:
# підключимо стандартизатор
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
_ = scaler.fit(X_numeric)

In [ ]:
(scaler.mean_, scaler.scale_, scaler.var_)

In [ ]:
X_numeric_scaled = pd.DataFrame(scaler.transform(X_numeric), columns=X_numeric_columns)

In [ ]:
X_numeric_scaled.describe()

### Перетворення категоріальних даних

[Докладніше про стратегії кодування](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)

In [ ]:
categorical_selector = make_column_selector(dtype_include='O')
X_categorical_columns = categorical_selector(X)
X_categorical_columns

In [ ]:
X['region'].unique()

In [ ]:
X_categorical = X[X_categorical_columns]
X_categorical.describe()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
_ = encoder.fit(X_categorical)

In [ ]:
X_categorical_encoded = encoder.transform(X_categorical)
X_categorical_encoded = pd.DataFrame(X_categorical_encoded, columns=encoder.categories_)

### Об'єднати числовий та категоріальний датасети

In [ ]:
(X_numeric_scaled.shape, X_categorical_encoded.shape)

In [ ]:
X = pd.concat([X_numeric_scaled, X_categorical_encoded], axis=1)

In [ ]:
X.head()

## ІНДИВІДУАЛЬНЕ ЗАВДАННЯ

In [ ]:
import warnings
warnings.filterwarnings('ignore')

<div style='background-color:lightgreen'>
<H3 style='color:red'>Виконати процедури очищення, збагачення та нормалізації даних рієлторської компанії в файлі `aprt_properties.csv`</H3>
<p>Н<b>а місці `...` написати свій код<b></p>
</div>


In [ ]:
# імпортувати бібліотеку Pandas, NumPy 
import pandas as pd
import numpy as np

### I. Вилучення сирих даних

__Примітка__: використати функцію `read_csv` з параметром `na_values=['None']` щоб згенерувати правильні типи даних

In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv("aprt_properties.csv", na_values=['None'])

ModuleNotFoundError: No module named 'pandas'

### II. Очищення (_опціонально_)

__Примітка__: на цьому етапі слід
1. Визначитись які показчики є категоріальними або числовими
2. При необхідності зробити перетворення типів (функція `astype`)
3. Визначити чи є в датасеті викиди.
4. При наявності викидів зробити їх корегування (`mean` або `median`)

In [2]:

print(df.describe().loc['count'])

mean_std = df.describe().loc[['mean', 'std']].T
mean_std['std_minus_mean'] = mean_std['std'] - mean_std['mean']

print(mean_std)
print(df)

for column in df[1:]:
    if(df[column].dtypes == 'float64'):
        df[column].fillna(value=df[column].median(), inplace=True)
    else:
        df[column].fillna(method='bfill', inplace=True)
        
df = df.astype({"rooms":'int32',"levels":'int32',"level":'int32',"year":'int32'})


two_sigma_price = 2 * mean_std.loc['Price']['std']
two_sigma_price_per_m2 = 2 * mean_std.loc['price_per_m2']['std']
two_sigma_area_living = 2 * mean_std.loc['area_living']['std']

print(two_sigma_price)
print(two_sigma_price_per_m2)
print(two_sigma_area_living)

NameError: name 'pd' is not defined

### III. Заповнення пропусків для чисельних показчиків

__Примітка__: на цьому етапі слід

1. Виділити датасет з чисельними показчиками використовуючи `make_column_selector`
2. Заповнити пусті (`NaN`) значення медіанними значеннями відповідної колонки (`fillna`)

In [ ]:
from sklearn.compose import make_column_selector

fractional_columns_selector = make_column_selector(dtype_include='float64')
target_columns_selector = make_column_selector(dtype_include='int32')

df_fractional_columns = fractional_columns_selector(df)
df_target_columns = target_columns_selector(df)

ds_fractional = df[df_numeric_columns].copy(deep=False)
ds_target = df[df_target_columns].copy(deep=False)

print(ds_fractional)
print(ds_target)

for fractional_column in ds_fractional:
    ds_fractional[fractional_column].fillna(value=ds_fractional[fractional_column].median(), inplace=True)

print(ds_fractional)
    
for target_column in ds_target:
    ds_target[target_column].fillna(value=ds_target[target_column].median(), inplace=True)


print(ds_target)

### IV. Масштабування чисельних показчиків

__Примітка__: на цьому етапі слід побудувати змаштабований датасет `df_num_scaled` з чисельних показчиків використавші модуль __`StandardScaler`__ та його функцію `fit_transform`

In [ ]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_fractional_scaled = scaler.fit_transform(ds_fractional)
df_target_scaled = scaler.fit_transform(ds_target)

print(f"{df_num_scaled}\n")
print(f"{df_target_scaled}\n")


### V. Заповнення пропусків для категоріальних показчиків

__Примітка__: на цьому етапі слід
1. Виділити датасет з категоріальними показчиками використовуючи `make_column_selector` з параметром `dtype_include='O'`
2. Видилити колонку `publish_date` бо вона в цьому датасеті чисто інформативна
3. Заповнити пусті (NaN) значення найближчими (`method='ffill'` в функції `fillna`)

In [7]:
from sklearn.compose import make_column_selector

df_columns_selector = make_column_selector(dtype_include='O')
df_columns = df_columns_selector(df)

ds = df[df_columns].copy(deep=False)
print(ds)

ds = ds.drop(columns=['publish_date'])
print(ds)

ds['street'].fillna(method='bfill', inplace = True)
print(ds)

ModuleNotFoundError: No module named 'sklearn'

### VI. Кодування категориальних показчиків

__Примітка__: на цьому етапі слід побудувати датасет `df_cat_encoded` використавши функцію  `fit_transform` модуля __`OneHotEncoder`__


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
import numpy as np

df_cat_encoded = encoder.fit_transform(ds)

print(df_cat_encoded)

### VII. Формування результатного датасету

__Примітка__: на цьому етапі слід об'єднати змаштабований чисельний (`df_num_scaled`) та кодований категоріальний (`df_cat_encoded`) датасети в один результатний `df_result` використавши функцію `concat`

In [8]:
ds_fractional_scaled = pd.DataFrame(df_fractional_scaled)
ds_target_scaled = pd.DataFrame(df_target_scaled)
ds_cat_encoded = pd.DataFrame(df_cat_encoded)

print(ds_fractional_scaled)
print(ds_target_scaled)
print(ds_cat_encoded)

df_result_frac = pd.concat([ds_fractional_scaled, ds_cat_encoded], axis=1)
df_result_tar = pd.concat([ds_target_scaled, ds_cat_encoded], axis=1)

print(df_result_frac)
print(df_result_tar)


NameError: name 'pd' is not defined